In [1]:
### See original: https://github.com/christophschuhmann/improved-aesthetic-predictor/blob/main/simple_inference.py
from PIL import Image
from warnings import filterwarnings
import numpy as np
import torch
import torch.nn.functional as F
import pytorch_lightning as pl
import torch.nn as nn
from transformers import CLIPProcessor, CLIPModel, AutoProcessor, AutoModel
from PIL import Image

In [2]:
class MLP(pl.LightningModule):
    def __init__(self, input_size, xcol='emb', ycol='avg_rating'):
        super().__init__()
        self.input_size = input_size
        self.xcol = xcol
        self.ycol = ycol
        self.layers = nn.Sequential(
            nn.Linear(self.input_size, 1024),
            nn.Dropout(0.2),
            nn.Linear(1024, 128),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.Dropout(0.1),
            nn.Linear(64, 16),
            nn.Linear(16, 1)
        )

    def forward(self, x):
        return self.layers(x)

    def training_step(self, batch, batch_idx):
            x = batch[self.xcol]
            y = batch[self.ycol].reshape(-1, 1)
            x_hat = self.layers(x)
            loss = F.mse_loss(x_hat, y)
            return loss
    
    def validation_step(self, batch, batch_idx):
        x = batch[self.xcol]
        y = batch[self.ycol].reshape(-1, 1)
        x_hat = self.layers(x)
        loss = F.mse_loss(x_hat, y)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

def normalized(a, axis=-1, order=2):
    ### Original, for numpy.ndarray input I think?
    # l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    # l2[l2 == 0] = 1
    # return a / np.expand_dims(l2, axis)
    
    ### For torch.Tensor input (written by neliopou)
    l2 = torch.norm(a, p=order, dim=axis, keepdim=True)
    l2[l2 == 0] = 1
    return a / l2


In [ ]:
class AestheticMLP(nn.Module):
    def __init__(self, input_size, xcol='emb', ycol='avg_rating'):
        super().__init__()
        self.input_size = input_size
        self.xcol = xcol
        self.ycol = ycol
        self.layers = nn.Sequential(
            nn.Linear(self.input_size, 1024),
            nn.Dropout(0.2),
            nn.Linear(1024, 128),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.Dropout(0.1),
            nn.Linear(64, 16),
            nn.Linear(16, 1)
        )

    def forward(self, x):
        return self.layers(x)

In [3]:
### Params
device = "cuda" if torch.cuda.is_available() else "cpu"

### Load CLIP ViT-L/14 Model
processor = AutoProcessor.from_pretrained("openai/clip-vit-large-patch14-336")
clip_model = AutoModel.from_pretrained("openai/clip-vit-large-patch14-336")
clip_model.to(device).eval()

### Load Aesthetic Score predictor
aesthetic_model = MLP(768)
aesthetic_model.load_state_dict( torch.load("sac+logos+ava1-l14-linearMSE.pth") )
aesthetic_model.to(device).eval()

images = [Image.open("diffusion_dog_iter0.png")]

image_inputs = processor(
   images=images,
   padding=True,
   truncation=True,
   max_length=16,
   return_tensors="pt",
).to(device)


C:\Users\Nick J. Eliopoulos\AppData\Local\Temp\ipykernel_33844\3782863335.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  aesthetic_model.load_state_dict( torch.load("s

In [4]:

with torch.no_grad():
	image_embs = clip_model.get_image_features(**image_inputs)
	image_embs = normalized(image_embs)
	prediction = aesthetic_model(image_embs).to(device)

print( prediction )

tensor([[4.9055]], device='cuda:0')
